In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Audio
%matplotlib inline
plt.rcParams['figure.dpi']= 150

First, we define parameters:

In [2]:
# Transport parameters
s_perm = 0.1
u_perm = 0.1
w_perm_D = 0.1
w_perm_C = 0.1
S_input = 1
U_input = 1
osm_input = 2
p_D = 0.1
p_C = 0.1

# discretization
N = 10
dx = 1/N

Now, we define the difference quotient operator

In [3]:
def diff_x(y): 
    n = len(y)-1
    diff = np.zeros(n)
    for l in range(n):
        diff[l] = (y[l+1]-y[l])/dx
    
    return diff

def avg(y):
    n = len(y)-1
    average = np.zeros(n)
    for l in range(n):
        average[l] = (y[l+1]+y[l])/2
    
    return average

In [4]:
def res(y):
    q_D = y[:(N+1)].copy()
    q_A = y[(N+1):2*(N+1)].copy()
    q_C = y[2*(N+1):3*(N+1)].copy()
    q_0 = y[3*(N+1):4*(N+1)].copy()
    s_D = y[4*(N+1):5*(N+1)].copy()
    s_A = y[5*(N+1):6*(N+1)].copy()
    s_0 = y[6*(N+1):7*(N+1)].copy()
    u_C = y[7*(N+1):8*(N+1)].copy()
    u_0 = y[8*(N+1):9*(N+1)].copy()

    residue = np.zeros(9*(N+1))
    res_D = residue[:(N+1)]
    res_A = residue[(N+1):2*(N+1)]
    res_C = residue[2*(N+1):3*(N+1)]
    res_0 = residue[3*(N+1):4*(N+1)]
    res_s_D = residue[4*(N+1):5*(N+1)]
    res_s_A = residue[5*(N+1):6*(N+1)]
    res_s_0 = residue[6*(N+1):7*(N+1)]
    res_u_C = residue[7*(N+1):8*(N+1)]
    res_u_0 = residue[8*(N+1):9*(N+1)]

    w_D = w_perm_D*(
            # (2*avg(s_0)+avg(u_0))-2*avg(s_D) + p_D
            (2*avg(s_0)+avg(u_0))/avg(q_0)-2*avg(s_D)/avg(q_D) + p_D
        )
    w_C = w_perm_C*(
            # (2*avg(s_0)+avg(u_0))-2*avg(u_C) + p_C
            (2*avg(s_0)+avg(u_0))/avg(q_0)-2*avg(u_C)/avg(q_C) + p_C
        )

    res_D[0] = q_D[0] - 2*S_input/osm_input
    res_D[1:] = diff_x(q_D) + w_D
    res_A[0] = q_A[-1] + q_D[-1]
    res_A[1:] = diff_x(q_A)
    res_C[0] = q_C[0] - U_input/osm_input
    res_C[1:] = diff_x(q_C) + w_C
    res_0[0] = q_0[-1]**2
    res_0[1:] = diff_x(q_0) - w_D - w_C

    # j_s = s_perm*(avg(s_A) - avg(s_0))
    j_s = s_perm*(avg(s_A)/avg(q_A) - avg(s_0)/avg(q_0))

    # res_s_D[0] = s_D[0] - osm_input/2
    res_s_D[0] = s_D[0] - S_input
    # res_s_D[1:] = diff_x(s_D*q_D)
    res_s_D[1:] = diff_x(s_D)
    res_s_A[0] = (2*S_input + U_input)/osm_input + q_0[0] + q_A[0] - q_C[-1]
    # res_s_A[1:] = diff_x(s_A*q_A) + j_s
    res_s_A[1:] = diff_x(s_A) + j_s
    # res_s_0[0] = S_input + q_A[0]*s_A[0] + s_0[0]*q_0[0]
    res_s_0[0] = S_input + s_A[0] + s_0[0]
    # res_s_0[1:] = diff_x(s_0*q_0) - j_s
    res_s_0[1:] = diff_x(s_0) - j_s

    j_u = u_perm*(avg(u_C)/avg(q_C) - avg(u_0)/avg(q_0))

    # res_u_C[0] = u_C[0] - osm_input
    res_u_C[0] = u_C[0] - U_input
    # res_u_C[1:] = diff_x(u_C*q_C) + j_u
    res_u_C[1:] = diff_x(u_C) + j_u
    # res_u_0[0] = U_input + u_0[0]*q_0[0] - u_C[-1]*q_C[-1]
    res_u_0[0] = U_input + u_0[0] - u_C[-1]
    # res_u_0[1:] = diff_x(u_0*q_0) - j_u
    res_u_0[1:] = diff_x(u_0) - j_u

    return residue

Finally, we put everything together:

In [5]:
# Transport parameters
s_perm = 0.1
u_perm = 0.1
w_perm_D = 0.1
w_perm_C = 0.1
S_input = 0.1
U_input = 1
osm_input = 2
p_D = 0
p_C = 0

# discretization
N = 20
dx = 1/N

#Initial guess
y_init = np.zeros(9*(N+1))
q_D_init = y_init[:(N+1)]
q_A_init = y_init[(N+1):2*(N+1)]
q_C_init = y_init[2*(N+1):3*(N+1)]
q_0_init = y_init[3*(N+1):4*(N+1)]
s_D_init = y_init[4*(N+1):5*(N+1)]
s_A_init = y_init[5*(N+1):6*(N+1)]
s_0_init = y_init[6*(N+1):7*(N+1)]
u_C_init = y_init[7*(N+1):8*(N+1)]
u_0_init = y_init[8*(N+1):9*(N+1)]

for n in range(N+1):
    q_D_init[n] = (1- n/(2*N))*2*S_input/osm_input
    q_C_init[n] = (1- n/(2*N))*U_input/osm_input
    q_0_init[n] = - 0.25*(1 - n/N)*(2*S_input+U_input)/osm_input

    # s_D_init[n] = (n/(2*N) + 1)*osm_input/2
    s_D_init[n] = q_D_init[n]*(n/(2*N) + 1)*osm_input/2
    s_A_init[n] = q_A_init[n]*(n/(2*N) + 1)*osm_input/2
    s_0_init[n] = q_0_init[n]*(n/(2*N) + 1)*osm_input/2

    u_C_init[n] = q_C_init[n]*(n/(2*N) + 1)*osm_input
    u_0_init[n] = q_0_init[n]*(n/(2*N) + 1)*osm_input

q_A_init[:] = - q_D_init[-1].copy()

y = fsolve(res,y_init)
q_D = y[:(N+1)]
q_A = y[(N+1):2*(N+1)]
q_C = y[2*(N+1):3*(N+1)]
q_0 = y[3*(N+1):4*(N+1)]
s_D = y[4*(N+1):5*(N+1)]
s_A = y[5*(N+1):6*(N+1)]
s_0 = y[6*(N+1):7*(N+1)]
u_C = y[7*(N+1):8*(N+1)]
u_0 = y[8*(N+1):9*(N+1)]
osm = (2*s_0 + u_0)/q_0